# Import requirements

In [1]:
import os
import random
import numpy as np
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(f'Device: {DEVICE}')


Device: cuda


In [2]:
!nvidia-smi

Tue Jul 27 21:06:39 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8    10W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Download the CIFAR-100 data

In [26]:
train_dataset = datasets.CIFAR100(root='../data/CIFAR100',
    train=True,
    download=True,
)
test_dataset = datasets.CIFAR100(root='../data/CIFAR100',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ]),
)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
for image, label in train_dataset:
    print(image.shape)
    print(label)
    break

for image, label in test_dataset:
    print(image.shape)
    print(label)
    break

torch.Size([3, 32, 32])
19
torch.Size([3, 32, 32])
49


In [27]:
train_list = []
for (image, label) in tqdm(train_dataset):
    img0 = image
    img1 = transforms.RandomHorizontalFlip(1)(image)
    img2 = transforms.RandomRotation(10)(image)
    img3 = transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2))(image)
    img4 = transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2)(image)
    imgs = [img0, img1, img2, img3, img4]
    
    for img in imgs:
        img = transforms.ToTensor()(img).view(-1, 32, 32)
        img = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(img)
        train_list.append((img, label))

print(f'\n{len(train_list)}')

100%|██████████| 50000/50000 [01:19<00:00, 631.87it/s]


250000


In [28]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, data_list):
        self.data = data_list
    
    def __len__(self):
        # size is same as length of list
        return len(self.data)

    def __getitem__(self, idx):
        # reshape tensor as (3, 32, 32)
        image = self.data[idx][0] #.view(3, 32, 32) 
        label = self.data[idx][1]
        return image, label

train_dataset = MyDataset(train_list)

In [29]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset, 
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4,
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4,
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


# Define the Convolution Neural Network (CNN)

In [30]:
class ConvNet(nn.Module):
    def __init__(self):
        # (3, 32, 32)
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), padding=(1, 1))
        self.bn1 = nn.BatchNorm2d(64)
        self.conv2 = nn.Conv2d(64, 128, kernel_size=(3, 3), padding=(1,1))
        self.bn2 = nn.BatchNorm2d(128)
        self.conv3 = nn.Conv2d(128, 128, kernel_size=(3, 3), padding=(1,1))
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=(3, 3), padding=(1,1))
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(256, 512, kernel_size=(3, 3), padding=(1,1))
        self.bn5 = nn.BatchNorm2d(512)
        self.conv6 = nn.Conv2d(512, 512, kernel_size=(3,3), padding=(1,1))
        self.bn6 = nn.BatchNorm2d(512)
        self.conv7 = nn.Conv2d(512, 1024, kernel_size=(3,3), padding=(1,1))
        self.bn7 = nn.BatchNorm2d(1024)
        self.conv8 = nn.Conv2d(1024, 1024, kernel_size=(3,3), padding=(1,1))
        self.bn8 = nn.BatchNorm2d(1024)

        self.pool = nn.MaxPool2d(2, 2)

        self.classify = nn.Sequential(
            nn.Linear(1024, 100),
        )

    def forward(self, x):
        # input shape: (3, 32, 32)
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = self.pool(x) # (128, 16, 16)
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = self.pool(x) # (256, 8, 8)
        x = F.relu(self.bn5(self.conv5(x)))
        x = self.pool(x) # (512, 4, 4)
        x = F.relu(self.bn6(self.conv6(x)))
        x = F.relu(self.bn7(self.conv7(x)))
        x = self.pool(x) # (1024, 2, 2)
        x = F.relu(self.bn8(self.conv8(x)))
        x = F.relu(self.bn8(self.conv8(x)))
        x = self.pool(x) # (1024, 1, 1)

        x = x.view(x.size(0), -1) # 1024
        x = self.classify(x)
        return x

# Define the train, evaluation

In [31]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    train_loss = 0
    correct = 0

    for batch_idx, (image, label) in enumerate(train_loader):
        image = image.to(DEVICE)
        label = label.to(DEVICE)

        optimizer.zero_grad()
        output = model(image)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

        if (batch_idx + 1) % log_interval == 0:
            pct = 100 * batch_idx / len(train_loader) # percent
            train_loss /= log_interval
            print(f'Train Epoch: {Epoch} [{batch_idx * len(image)}/{len(train_loader.dataset)} ({pct:.0f}%)]\tAverage Train Loss: {train_loss:.6f}')
            train_loss = 0


def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for image, label in test_loader:
            image = image.to(DEVICE)
            label = label.to(DEVICE)
            
            output = model(image)
            test_loss += criterion(output, label).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(label.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100 * correct / len(test_loader.dataset)

    return test_loss, test_accuracy

# set seeds

In [32]:
def fix_seeds(seed = 42, use_torch=False):
    # fix the seed for reproducibility 
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)

    if use_torch: 
        torch.manual_seed(seed) 
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True

# initialize the weights

In [33]:
def init_weights(m):
    # initialize the weight, bias
    if isinstance(m, nn.Conv2d):
        torch.nn.init.kaiming_uniform_(m.weight.data)
        if m.bias is not None:
            torch.nn.init.normal_(m.bias.data)
    elif isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight.data, mean=1, std=0.02)
        torch.nn.init.constant_(m.bias.data, 0)
    elif isinstance(m, nn.Linear):
        torch.nn.init.kaiming_uniform_(m.weight.data)
        torch.nn.init.normal_(m.bias.data)

# Train & Test the model

In [34]:
SEED = 42
EPOCHS = 100

fix_seeds(seed=SEED, use_torch=True)
model = ConvNet().to(device=DEVICE)
model.apply(init_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)
criterion = nn.CrossEntropyLoss()


for Epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer, log_interval=int(len(train_loader) * 0.2))
    test_loss, test_acc = evaluate(model, test_loader)
    scheduler.step(test_loss)

    print(f'\nEpoch: {Epoch}')
    print(f'Average Test Loss: {test_loss:.4f}')
    print(f'Test Accuracy: {test_acc:.2f} %\n')
    #torch.save(model, f'./models/model_{Epoch:02d}.pt')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Train Epoch: 1 [49952/250000 (20%)]	Average Train Loss: 4.174800
Train Epoch: 1 [99936/250000 (40%)]	Average Train Loss: 3.076969
Train Epoch: 1 [149920/250000 (60%)]	Average Train Loss: 2.473030
Train Epoch: 1 [199904/250000 (80%)]	Average Train Loss: 2.129765
Train Epoch: 1 [249888/250000 (100%)]	Average Train Loss: 1.833055

Epoch: 1
Average Test Loss: 0.1112
Test Accuracy: 25.03 %

Train Epoch: 2 [49952/250000 (20%)]	Average Train Loss: 1.561672
Train Epoch: 2 [99936/250000 (40%)]	Average Train Loss: 1.395788
Train Epoch: 2 [149920/250000 (60%)]	Average Train Loss: 1.249112
Train Epoch: 2 [199904/250000 (80%)]	Average Train Loss: 1.118763
Train Epoch: 2 [249888/250000 (100%)]	Average Train Loss: 0.993196

Epoch: 2
Average Test Loss: 0.1084
Test Accuracy: 33.67 %

Train Epoch: 3 [49952/250000 (20%)]	Average Train Loss: 0.724231
Train Epoch: 3 [99936/250000 (40%)]	Average Train Loss: 0.670889
Train Epoch: 3 [149920/250000 (60%)]	Average Train Loss: 0.601991
Train Epoch: 3 [199904/250

KeyboardInterrupt: ignored